In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Configuration de l'affichage pour voir toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Librairies chargées avec succès !")

Librairies chargées avec succès !


In [4]:
# Chargement de la matrice et des catalogues
print("--- Chargement des données ---")
X = load_npz('../out/user_permission_matrix_sparse.npz')
users_df = pd.read_csv('../out/users_catalog.csv')
perms_df = pd.read_csv('../out/perm_catalog.csv')
apps_df = pd.read_csv('../out/app_catalog.csv')

print(f"Matrice chargée : {X.shape} (Utilisateurs x Permissions)")
print(f"Nombre d'utilisateurs : {len(users_df)}")
print(f"Nombre de permissions : {len(perms_df)}")

--- Chargement des données ---
Matrice chargée : (5000, 324) (Utilisateurs x Permissions)
Nombre d'utilisateurs : 5000
Nombre de permissions : 324


In [ ]:
# --- Configuration NMF ---
N_ROLES = 30  # Assez grand pour capter les métiers ET les villes
print(f"Lancement de la NMF pour découvrir {N_ROLES} rôles...")

model_nmf = NMF(n_components=N_ROLES, init='nndsvd', random_state=42, max_iter=1000)

# W : Matrice Utilisateur x Rôles (Poids)
W = model_nmf.fit_transform(X)

# H : Matrice Rôles x Permissions (Composition)
H = model_nmf.components_

# --- NORMALISATION ---
# Important : On convertit les poids bruts en pourcentages relatifs
# Pour que la somme des poids d'un utilisateur soit égale à 1
W_norm = W / W.sum(axis=1, keepdims=True)
W_norm = np.nan_to_num(W_norm) # Gestion des divisions par zéro

print("NMF terminée !")
print(f"Matrice des profils utilisateurs (W_norm) : {W_norm.shape}")

In [ ]:
# --- DÉFINITION DU SEUIL ---
# Si un utilisateur a plus de 10% de son "ADN" dans un rôle, on considère qu'il l'a.
THRESHOLD = 0.10

# Y_binary est une matrice de 0 et 1.
# Chaque ligne correspond à un utilisateur, chaque colonne à un rôle.
# Exemple : [1, 0, 0, 1, ...] signifie "Possède le Rôle 0 et le Rôle 3"
Y_binary = (W_norm > THRESHOLD).astype(int)

# Vérification statistique
avg_roles = np.mean(np.sum(Y_binary, axis=1))
print(f"--- Statistiques Multi-Rôles ---")
print(f"Avec un seuil de {THRESHOLD*100}%, un utilisateur possède en moyenne {avg_roles:.2f} rôles.")
print("Si ce chiffre est proche de 1, baisse le seuil. S'il est > 5, monte le seuil.")

In [ ]:
print("--- Préparation et Entraînement Supervisé ---")

# 1. Encodage des entrées (X)
le_dept = LabelEncoder()
users_df['dept_encoded'] = le_dept.fit_transform(users_df['department'])

le_pos = LabelEncoder()
users_df['pos_encoded'] = le_pos.fit_transform(users_df['position'])

le_loc = LabelEncoder()
users_df['loc_encoded'] = le_loc.fit_transform(users_df['location'])

X_features = users_df[['dept_encoded', 'pos_encoded', 'loc_encoded']]

# 2. Séparation Train / Test
X_train, X_test, y_train, y_test = train_test_split(X_features, Y_binary, test_size=0.2, random_state=42)

# 3. Entraînement
# Random Forest gère nativement le "Multi-Output" (prédire plusieurs 1 à la fois)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 4. Évaluation rapide
y_pred = clf.predict(X_test)
# L'accuracy exacte est sévère (il faut avoir TOUT bon sur la ligne), donc on regarde un score moyen
print(f"Modèle entraîné sur {len(X_train)} utilisateurs.")
print(f"Précision 'Subset' (Tout bon ou rien) : {accuracy_score(y_test, y_pred):.2%}")

In [ ]:
def describe_role(role_id):
    """Fonction helper pour donner un nom lisible à un rôle ID"""
    # On regarde les 3 apps principales du rôle pour comprendre ce que c'est
    top_idx = H[role_id].argsort()[::-1][:3]
    top_apps = perms_df.iloc[top_idx].merge(apps_df, on='application_id')['app_name'].unique()
    return f"Rôle {role_id} (ex: {', '.join(top_apps)})"

print("--- SIMULATION : PROVISIONING AUTOMATIQUE ---")

# --- PARAMÈTRES DU NOUVEAU VENU ---
NEW_DEPT = "Department Sales"
NEW_POS  = "Sales Manager"
NEW_LOC  = "Lyon"

print(f"Arrivée d'un : {NEW_POS} / {NEW_DEPT} / {NEW_LOC}")

try:
    # 1. Encodage des infos
    encoded_input = [[
        le_dept.transform([NEW_DEPT])[0],
        le_pos.transform([NEW_POS])[0],
        le_loc.transform([NEW_LOC])[0]
    ]]
    
    # 2. Prédiction
    predicted_vector = clf.predict(encoded_input)[0] # Renvoie [0, 1, 0, 1, ...]
    
    # 3. Interprétation
    # On récupère les indices où il y a un '1'
    found_role_indices = np.where(predicted_vector == 1)[0]
    
    print(f"\n✅ L'IA suggère d'attribuer {len(found_role_indices)} rôles :")
    for r_id in found_role_indices:
        print(f"  -> {describe_role(r_id)}")

except Exception as e:
    print(f"Erreur : Une des valeurs (Ville/Poste) n'est pas connue du modèle entrainé. ({e})")